# Port von Better Seite

In [1]:
import pdal 
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import KDTree
from scipy.linalg import svd 
from osgeo import ogr
import pyvista as pv
import os
import sys
import open3d as o3d
import json
import time 
import matplotlib.pyplot as plt

from interessant import * # Bei Änderungen Kernel neu starten

sys.path.append("/home/riannek/code/gleis/gleisachse")
from algs.centralpoints import *
from algs.voxelfilter import *

from scipy.spatial.transform import Rotation as R
ogr.UseExceptions()

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
tmpdir = "/media/riannek/minimax/gleis/temp_run24"

# Bahnsteig: 29; Gleis hohe Intensität: 11; Weiche B: 16; Unterirdischer Bhf: 20; Gleis weit abseits: 23; Betondeckel: 28; Zug run 14 A: 6; 
# Viele Gleise: 33; Anfang Weiche: 34; OLA gleiche H: 35; Y: 37; Auch viele Gleise: 43; Kreuzung: 44, 45;

key = list(interessant.keys())[0] 

filename = interessant[key]


# filename = "4474550_5332275.copc.laz"   ###############################


filename = filename.split(".")[0] + ".ply"
print(key, filename)


if not os.path.exists(os.path.join(tmpdir, "candidates",filename)):
    raise FileNotFoundError(filename)



Einfach 4473900_5335875.ply


In [3]:
thresh_side_distance = 0.15   # 0.1 ist meist gut, aber in manchen Fällen zu wenig (z.B. Anfang Weiche)

intensity_threshold = 14500
downsample_radius = 0.45 # 0.4
neighborhood_radius = 0.5 # 0.5

min_points = 10
minimum_in_hood = 10
linearity_tresh = 0.98

gauge = 1.435
railhead_width = 0.067

In [4]:
runfolder = "/media/riannek/minimax/gleis/run24-2024-08-13"

import subprocess
# subprocess.Popen(["pyvistaviewer", os.path.join(runfolder, filename.split(".")[0] + ".copc.laz")])

## Datei 
oben incl. margin aus Nachbarkacheln, unten ohne

In [5]:
def extend_bbox(bbox, margin=2):
    # MinX, MaxX, MinY, MaxY
    return (bbox[0] - margin, bbox[1] + margin, bbox[2] - margin, bbox[3] + margin)

def get_bbox_polygon(bbox):  
    ring = ogr.Geometry(ogr.wkbLinearRing)      
    ring.AddPoint_2D(bbox[0], bbox[2])  # MinX, MinY
    ring.AddPoint_2D(bbox[1], bbox[2])  # MaxX, MinY
    ring.AddPoint_2D(bbox[1], bbox[3])  # MaxX, MaxY
    ring.AddPoint_2D(bbox[0], bbox[3])  # MinX, MaxY
    ring.AddPoint_2D(bbox[0], bbox[2])  # Close ring
    geom = ogr.Geometry(ogr.wkbPolygon)
    geom.AddGeometry(ring)
    return geom

In [6]:
# gpkg = ogr.Open(os.path.join(tmpdir, "temp.gpkg"))
# layer = gpkg.GetLayerByName("tiles")


# filter = f"filename = '{filename}'"
# layer.SetAttributeFilter(filter)
# feature = layer.GetNextFeature()
# layer.SetAttributeFilter(None)
# bbox = feature.GetGeometryRef().GetEnvelope()
# extended = extend_bbox(bbox, margin=2)
# bbox_geom = get_bbox_polygon(extended)
# layer.SetSpatialFilter(bbox_geom)
# tiles = [f.GetField("filename") for f in layer]
# layer.SetSpatialFilter(None) 

# del gpkg

# bounds = f"([{extended[0]}, {extended[1]}], [{extended[2]}, {extended[3]}])" 

In [7]:
# readers = [pdal.Reader(os.path.join(tmpdir, "candidates", tile)) for tile in tiles]
# pipeline = pdal.Pipeline(readers) | pdal.Filter.merge() | pdal.Filter.crop(bounds=bounds)
# pipeline.execute()
# points = pipeline.arrays[0]
# points.shape 

In [8]:
filename = os.path.join(tmpdir, "candidates", filename)
pipeline = pdal.Pipeline([pdal.Reader(filename)])
pipeline.execute()
points = pipeline.arrays[0]
points.shape 

(91051,)

## Seed Points

In [9]:
low_intensity = points[points["Intensity"] < intensity_threshold]
low_intensity.shape

(49162,)

In [10]:
# bounds = f"([{bbox[0]}, {bbox[1]}], [{bbox[2]}, {bbox[3]}])" 
# bounds

In [11]:
# Downsample with poisson sampling (only original bbox)

# downsampling_pipeline = pdal.Filter.crop(bounds=bounds).pipeline(low_intensity) | pdal.Filter("filters.sample", radius=downsample_radius)

downsampling_pipeline = pdal.Filter("filters.sample", radius=downsample_radius).pipeline(low_intensity)
downsampling_pipeline.execute()
seed_points = downsampling_pipeline.arrays[0]
seed_point_count = seed_points.shape[0]
seed_point_count

266

## KD Tree

In [12]:
xyz = np.vstack((points['X'], points['Y'], points['Z'])).transpose()
xyz_seed = np.vstack((seed_points['X'], seed_points['Y'], seed_points['Z'])).transpose()

In [13]:
offset = xyz.mean(axis=0) # .round() 
xyz -= offset
xyz_seed -= offset

In [14]:
tree = KDTree(xyz)  

# indices: ndarray (dtype object) with a list of indices for each seed point
indices = tree.query_ball_point(xyz_seed, r=neighborhood_radius, workers=-1)

## Alle

In [15]:
with_lines = True

In [16]:
def pv_bbox(plotter, bbox, offset=None):
    points = np.array(
        [[bbox[0], bbox[2], 0],
        [bbox[1], bbox[2], 0],
        [bbox[1], bbox[3], 0],
        [bbox[0], bbox[3], 0],
        [bbox[0], bbox[2], 0]]
    )
    if offset is not None:
        points[:,:2] -= offset[:2]
    lines = np.array([5, 0, 1, 2, 3, 0])
    box = pv.PolyData(points)
    box.lines = lines
    plotter.add_mesh(box, color="black", line_width=2, point_size=0, render_points_as_spheres=False)

In [17]:
start = time.time()

lines = []
center_points = []
delta_z = []
distances = []
representatives = []


upperpercentile=90
lowerpercentile = 70

for cluster, clustercenter, cluster_other, center_other, eigenvects in pair_generator(xyz, tree, indices, seed_point_count, gauge):
    transformmatrix = get_transformmatrix(cluster, cluster_other, clustercenter, center_other, eigenvects)
    representative_a, side_a  = representative_point(cluster, transformmatrix, railhead_width=0.067) 
    representative_b, side_b = representative_point(cluster_other, transformmatrix, railhead_width=0.067)

    if representative_a is None or representative_b is None:
        # Happens in rare cases at switches
        continue
    
    # Check if both are classified as the same side
    # but a zero and non zero value is not a problem
    if side_a * side_b < 0:
        # Update the cluster with lower confidence of the side estimation
        if np.abs(side_a) < np.abs(side_b):
            representative_a, side_a  = representative_point(cluster, transformmatrix, railhead_width=0.067, side=side_b)
        else:
            representative_b, side_b = representative_point(cluster_other, transformmatrix,  railhead_width=0.067, side=side_a)

    if representative_a is None or representative_b is None:
        # Happens in rare cases at switches
        continue

    central = get_central_point(representative_a, representative_b)
    lines.append(pv.Line(representative_a, representative_b))
    center_points.append(central)
    delta_z.append(np.abs(representative_a[2] - representative_b[2]))
    distances.append(np.linalg.norm(representative_a[:2] - representative_b[:2]))
    representatives.append(representative_a)
    representatives.append(representative_b)

timedelta = time.time() - start
print(f"Time: {timedelta:.3f} s per {len(center_points)} = {timedelta/len(center_points):.4f} s")

Time: 0.709 s per 168 = 0.0042 s


In [18]:
pv_all = pv.PolyData(xyz)
pv_seed = pv.PolyData(xyz_seed) 

In [19]:
center_points = np.array(center_points)
rep_points = np.array(representatives)

pv_centerpoints = pv.PolyData(center_points)
pv_centerpoints["z"] = center_points[:,2]
pv_centerpoints["diff_z"] = np.abs(np.array(delta_z))
pv_centerpoints["distance"] = np.array(distances)

pv_rep = pv.PolyData(rep_points)

p = pv.Plotter()

if with_lines:
    # Add lines (SLOW)
    # for color, line in zip(side_distances, lines):
    #     p.add_mesh(line, scalars=color, cmap='plasma')
    for line in lines:
        p.add_mesh(line, color="red")

p.add_mesh(pv_all, color='gray', point_size=2)
# p.add_mesh(pv_seed, color='green', point_size=3)
p.add_mesh(pv_centerpoints, scalars="distance", point_size=5, cmap="viridis")
p.add_mesh(pv_rep, color="red", point_size=3)
# pv_bbox(p, bbox, offset)

# labels = [str(i) for i in range(len(center_points))]
# p.add_point_labels(pv_centerpoints, labels, point_color="black", font_size=12)

p.show()

# Farbe: Punkte nach "z" oder "diff_z" oder "distance"

Widget(value='<iframe src="http://localhost:43823/index.html?ui=P_0x7fd9ec0d65e0_0&reconnect=auto" class="pyvi…

## Einzeln

In [20]:
pair_gen = pair_generator()

TypeError: pair_generator() missing 5 required positional arguments: 'xyz', 'tree', 'indices', 'seed_point_count', and 'gauge'

In [ ]:
# for i in range(122):
#     next(pair_gen)

In [ ]:
np.argmin(np.abs(np.array([0.2, -0.8])))

np.int64(0)

In [ ]:
cluster, clustercenter, cluster_other, clustercenter_other, eigenvects = next(pair_gen)

percentile = 95

upperpercentile=90
lowerpercentile = 70

transformmatrix = get_transformmatrix(cluster, cluster_other, clustercenter, clustercenter_other, eigenvects)
#print("Transformmatrix")
#print(transformmatrix)

if transformmatrix[2,2] < 0: # z of the third eigenvector
    print("Inverted eigenvectors")
    transformmatrix *= -1
    


representative_aaa, side_a  = representative_point_3(cluster, transformmatrix, upperpercentile=upperpercentile, lowerpercentile=lowerpercentile, railhead_width=0.067)
representative_bbb, side_b = representative_point_3(cluster_other, transformmatrix, upperpercentile=upperpercentile, lowerpercentile=lowerpercentile, railhead_width=0.067)

print("Sides", side_a, side_b)

# Check if both are classified as the same side
# but a zero and non zero value is not a problem
if side_a * side_b < 0:
    print("Different sides")
    # Update the cluster with lower confidence of the side estimation
    if np.abs(side_a) < np.abs(side_b):
        representative_aaa, side_a  = representative_point_3(cluster, transformmatrix, upperpercentile=upperpercentile, lowerpercentile=lowerpercentile, railhead_width=0.067, side=side_b)
    else:
        representative_bbb, side_b = representative_point_3(cluster_other, transformmatrix, upperpercentile=upperpercentile, lowerpercentile=lowerpercentile, railhead_width=0.067, side=side_a)


    #representative_aaa, representative_bbb = representative_bbb, representative_aaa

central = get_central_point(representative_a, representative_b)
# print("Central Point", central)
# print("representative Points:", representative_a, representative_b)

#o3d.visualization.draw_geometries([pcd_all, pcd_clusters, mesh_frame, pcd_clustercenters, pcd_representative])
print("Points per Cluster", len(cluster), len(cluster_other))
fig, ax, transformed, transformed_other = plot_cluster_pair_compare(cluster, cluster_other, transformmatrix, representative_a, representative_b, representative_aa, representative_bb, representative_aaa, representative_bbb, railhead_width = 0.067)

pair = np.vstack((cluster, cluster_other))
plot_cluster(pair, transformmatrix);
# Grün: Version 0; Blau: Version 1; Braun: Version 2

NameError: name 'representive_point_3' is not defined

In [ ]:
np.corrcoef(transformed[transformed[:,2]>0][:,0], transformed[transformed[:,2]>0][:,1])[0,1], np.corrcoef(transformed[transformed[:,2]<0][:,0], transformed[transformed[:,2]<0][:,1])[0,1]

(np.float64(-0.07571001225647442), np.float64(-0.07437805190401485))

In [ ]:
np.corrcoef(transformed_other[transformed_other[:,2]>0][:,0], transformed_other[transformed_other[:,2]>0][:,1])[0,1], np.corrcoef(transformed_other[transformed_other[:,2]<0][:,0], transformed_other[transformed_other[:,2]<0][:,1])[0,1]

(np.float64(0.005539811720963599), np.float64(-0.05649399095569635))

In [ ]:
pcd_clusters = o3d.geometry.PointCloud()
pcd_clusters.points = o3d.utility.Vector3dVector(pair)
pcd_clusters.paint_uniform_color([0, 0, 1])

pcd_clustercenters = o3d.geometry.PointCloud()
pcd_clustercenters.points = o3d.utility.Vector3dVector([clustercenter, clustercenter_other])
pcd_clustercenters.paint_uniform_color([0, 1, 0])

pcd_all = o3d.geometry.PointCloud()
pcd_all.points = o3d.utility.Vector3dVector(xyz)
pcd_all.paint_uniform_color([0.7, 0.7, 0.7])

pcd_representative = o3d.geometry.PointCloud()
pcd_representative.points = o3d.utility.Vector3dVector([representative_a, representative_b, central])
pcd_representative.paint_uniform_color([1, 0, 0])

mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.5, origin=[0, 0, 0])

# o3d.visualization.draw_geometries([pcd_all, pcd_clusters, mesh_frame, pcd_clustercenters, pcd_representative])

# Experimentieren

In [ ]:
nrpoints = 1803127 
seconds = 3.7 + 1.5 + 1.5

seconds * 1000000 / nrpoints 

3.7157671090278166

In [ ]:
nrpoints = 6000
seconds = 10/100 + 4/100

seconds * 1000000 / nrpoints 

23.333333333333332

In [ ]:
# Clusterbeispiel speichern

prefix = "Auch komisch verschiedene Haken und Luecke"

pair = np.vstack((cluster, cluster_other))

save_clusters(cluster, cluster_other, pair, prefix)

FileExistsError: File already exists